# Test SAFEtorch pipeline

In [ ]:
from utils.function_normalizer import FunctionNormalizer
from utils.instructions_converter import InstructionsConverter
from utils.capstone_disassembler import disassemble
from utils.radare_analyzer import BinaryAnalyzer
from safetorch.safe_network import SAFE
from safetorch.parameters import Config
import torch

In [ ]:
# initialize SAFE
config = Config()
safe = SAFE(config)

In [ ]:
# load instruction converter and normalizer
I2V_FILENAME = "model/word2id.json"
converter = InstructionsConverter(I2V_FILENAME)
normalizer = FunctionNormalizer(max_instruction=150)

In [ ]:
# load SAFE weights
SAFE_torch_model_path = "model/SAFEtorch.pt"
state_dict = torch.load(SAFE_torch_model_path)
safe.load_state_dict(state_dict)
safe = safe.eval()

In [ ]:
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
print('Cosine Similarity: ')
print(cos(main, main_fuffa))

# Create exploits_embeddings

In [ ]:
#change these variables according to your setup
#path to folder with exploits
exploit_folder = '/media/mattia/2068D30968D2DC9A/Users/matti/Desktop/Magistrale/tesi/poc_detection/temp_dir/'
#path to embeddings_extractor script
embeddings_extractor = 'embeddings_extractor.py'


In [ ]:
from utils.function_normalizer import FunctionNormalizer
from utils.instructions_converter import InstructionsConverter
from utils.capstone_disassembler import disassemble
from utils.radare_analyzer import BinaryAnalyzer
from safetorch.safe_network import SAFE
from safetorch.parameters import Config
import torch
import gc
gc.collect()
import os
filtered = filter(lambda elem: 'exe' in exploit_folder + elem, os.listdir(exploit_folder))
filtered = map(lambda elem: exploit_folder + elem, filtered)
binary_exploits = list(filtered)

# create dict with {exploit name, (function_name,embedding)}
exploits_embeddings = {}
    
for exe in binary_exploits:
    listone = !python $embeddings_extractor $exe
    if len(listone) > 0:
        exploits_embeddings[exe.replace('.exe','').replace(exploit_folder,'')] = listone
    print(exe.replace('.exe','').replace(exploit_folder,'') + ' processed')



## Remove invalid entries in exploits_embeddings

In [ ]:
# remove invalid entries in exploits_embeddings
from copy import deepcopy
d = deepcopy(exploits_embeddings)

for key in d:
    x =str(d[key])
    if 'exe' in x:
        del exploits_embeddings[key]

exploits_embeddings
# save exploits_embeddings as file
torch.save(exploits_embeddings,'/home/mattia/Desktop/tesi_magistrale/SAFEtorch/SAFEtorch/exploits_embeddings_complete.pt')

# Load exploits_embeddings from file

In [ ]:
exploits_embeddings = torch.load('/home/mattia/Desktop/tesi_magistrale/SAFEtorch/SAFEtorch/exploits_embeddings_complete.pt')
print('exploits_embeddings loaded')